In [15]:
%run 'utils.ipynb'

In [16]:
# data_dir = '/home/joan/PycharmProjects/titanic/data'
import os

data_dir = '../data'
train_file = os.path.join(data_dir, 'train.csv')

In [35]:
import pandas as pd

data = pd.read_csv(train_file)
# data = drop_cols(data, ['PassengerId', 'Name', 'Ticket', 'Cabin'])
data.head()
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
data = impute_data(data)
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [19]:
data.groupby(['Sex'])['Survived'].sum() * 100 /  data.groupby(['Sex'])['Survived'].count()

Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64

In [20]:
data.groupby(['Pclass'])['Survived'].sum() * 100 / data.groupby(['Pclass'])['Survived'].count()

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64

In [22]:
attr = 'SibSp'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

SibSp
0    34.539474
1    53.588517
2    46.428571
3    25.000000
4    16.666667
5     0.000000
8     0.000000
Name: Survived, dtype: float64

In [23]:
attr = 'Parch'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

Parch
0    34.365782
1    55.084746
2    50.000000
3    60.000000
4     0.000000
5    20.000000
6     0.000000
Name: Survived, dtype: float64

In [24]:
data = process_family(data)

attr = 'num_family'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

attr = 'is_alone'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()


is_alone
False    50.564972
True     30.353818
Name: Survived, dtype: float64

In [25]:
attr = 'Embarked'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

Embarked
C    55.357143
Q    38.961039
S    33.900929
Name: Survived, dtype: float64

In [26]:
# categorize age
data = categorize_age(data)
attr = 'age_cat'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

age_cat
(0.34, 16.336]      55.000000
(16.336, 32.252]    34.416826
(32.252, 48.168]    40.425532
(48.168, 64.084]    43.478261
(64.084, 80.0]       9.090909
Name: Survived, dtype: float64

In [27]:
# categorize fare
data = categorize_fare(data) 
attr = 'fare_cat'
data.groupby([attr])['Survived'].sum() * 100 / data.groupby([attr])['Survived'].count()

fare_cat
(-0.001, 7.91]     19.730942
(7.91, 14.454]     30.357143
(14.454, 31.0]     45.495495
(31.0, 512.329]    58.108108
Name: Survived, dtype: float64

In [28]:
cat_features = ['Sex', 'Embarked', 'is_alone', 'fare_cat', 'age_cat']
data = data_to_cat(data, cat_features)
data[cat_features].head()

,Sex,Embarked,is_alone,fare_cat,age_cat
0,1,2,0,0,1
1,0,0,0,3,2
2,0,2,1,1,1
3,0,2,0,3,2
4,1,2,1,1,2


In [32]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(data, test_size=0.2)

In [33]:
# define input features
# features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
features = ['Pclass', 'Sex', 'age_cat', 'fare_cat', 'is_alone', 'Embarked']

X = train[features]
y = train[['Survived']]
dev_X = dev[features]
dev_y = dev[['Survived']]

In [34]:
# read test data
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
passenger_id = test[['PassengerId']]
test = impute_data(test)
test = process_family(test)
test = categorize_age(test)
test = categorize_fare(test)
test = data_to_cat(test, cat_features)

test_X = test[features]
test_X.head()
# test_X.isnull().sum()

,Pclass,Sex,age_cat,fare_cat,is_alone,Embarked
0,3,1,2,0,1,1
1,3,0,3,0,0,2
2,2,1,4,1,1,1
3,3,1,1,1,1,2
4,3,0,1,1,0,2
